In [ ]:
"""Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.

Attributes
----------
instruments
transactions
portfolios
set holdings
output transactions
corporate actions
stock split
build transactions
"""

# Output Transactions Notebook


When LUSID returns a portfolio, holdings or valuation it runs all calculations over the timeline of transactions within a portfolio. 

This is straightforward when a portfolio consists of a simple list of user-added transactions but what happens when portfolios are subject to adjustment from something other than a transaction?

This notebook shows how LUSID creates synthetic transactions within portfolios to represent non-transaction events such as Corporate Actions (e.g. bonus issue, dividend, stock split) or Holdings Adjustments. 

These synthetic transactions can be seen in the Output Transactions at the end of the notebook.


## Part 1: Setting up LUSID

To initialise our LUSID environment run the cell below


In [2]:
# Import general purpose packages
import json
import os
import pandas as pd
from datetime import datetime

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.utilities import create_scope_id

# Set pandas options
pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# Create a scope and portfolio code
scope = create_scope_id()
portfolio_code = "EQUITY_UK"

### Load source transactions

We have a csv of transactions representing initial holdings of 10 FTSE100 stocks that we can use to initialise our portfolio. 

In [3]:
df = pd.read_csv("data/equity_transactions_for_output.csv")
df

portfolio_code                       portfolio_name  \
0       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
1       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
2       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
3       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
4       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
5       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
6       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
7       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
8       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
9       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
10      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
11      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
12      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
13      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
14      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
15      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
16      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
17      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
18      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
19      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   

   portfolio_base_currency        ticker    sedol instrument_type  \
0                      GBP  GB0002162385   SEDOL1          equity   
1                      GBP  GB0002162385   SEDOL1          equity   
2                      GBP  GB0000566504   SEDOL2          equity   
3                      GBP  GB0000566504   SEDOL2          equity   
4                      GBP  GB0031348658   SEDOL3          equity   
5                      GBP  GB0031348658   SEDOL3          equity   
6                      GBP  GB0007980591   SEDOL4          equity   
7                      GBP  GB0007980591   SEDOL4          equity   
8                      GBP  GB0005405286   SEDOL5          equity   
9                      GBP  GB0005405286   SEDOL5          equity   
10                     GBP  GB0006043169   SEDOL6          equity   
11                     GBP  GB0006043169   SEDOL6          equity   
12                     GBP  GB0008847096   SEDOL7          equity   
13                     GBP  GB0008847096   SEDOL7          equity   
14                     GBP  GB00BGDT3G23   SEDOL8          equity   
15                     GBP  GB00BGDT3G23   SEDOL8          equity   
16                     GBP  GB00BH4HKS39   SEDOL9          equity   
17                     GBP  GB00BH4HKS39   SEDOL9          equity   
18                     GBP  GB00B1XZS820  SEDOL10          equity   
19                     GBP  GB00B1XZS820  SEDOL10          equity   

   instrument_id                name    txn_id txn_type txn_trade_date  \
0        EQ_1234               Aviva  trd_0001  StockIn     02/01/2020   
1        EQ_1234               Aviva  trd_0002  StockIn     02/01/2020   
2        EQ_1235                 BHP  trd_0003  StockIn     02/01/2020   
3        EQ_1235                 BHP  trd_0004  StockIn     02/01/2020   
4        EQ_1236            Barclays  trd_0005  StockIn     02/01/2020   
5        EQ_1236            Barclays  trd_0006  StockIn     02/01/2020   
6        EQ_1237                  BP  trd_0007  StockIn     02/01/2020   
7        EQ_1237                  BP  trd_0008  StockIn     02/01/2020   
8        EQ_1238                HSBC  trd_0009  StockIn     02/01/2020   
9        EQ_1238                HSBC  trd_0010  StockIn     02/01/2020   
10       EQ_1239           Morrisons  trd_0011  StockIn     15/01/2020   
11       EQ_1239           Morrisons  trd_0012  StockIn     15/01/2020   
12       EQ_1240               Tesco  trd_0022  StockIn     15/01/2020   
13       EQ_1240               Tesco  trd_0014  StockIn     16/01/2020   
14       EQ_1241           Rightmove  trd_0015  StockIn     15/01/2020   
15       EQ_1241           Rightmove  trd_0016  StockIn     15/01/2020   
16       EQ_1242            vodafone  trd_0017  StockIn     15/01/2020   
17

### Load instruments

Using the csv of setup transactions we can create instruments in LUSID for each of the equities referenced. When uploading the instruments we need to map certain properties that LUSID expects to columns provided in the csv.

For example, we need to map the 'ticker' column to the 'Isin' identifier and then 'sedol' to 'Sedol'.

In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ticker",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
}

In [5]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

# Check the success rate of the upload
succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0       10       0       0

### Create portfolio

To create the portfolio we will use the transaction csv as we did when creating instruments, mapping LUSID's required properties to columns in the dataframe, also providing a name, base currency and (optionally) setting the date at which the portfolio should be effective from (created).

In [6]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [7]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

# Check the success rate of the upload
succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        1       0       0

### Load in transactions

Now that we have a list of instruments and a portfolio setup in LUSID, we can complete the initialisation by uploading the transactions.

The approach is similar to the instruments and portfolio but there is a slightly more extensive mapping exercise to complete.

In [8]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",        
    },
    "optional": {},
    "properties": [],
}

In [9]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

# Check the success rate of the upload
succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(failed)}])

success  failed  errors
0        1       0       0

### Check holdings

Now that we have initialised the portfolio, we can query LUSID to check our holdings.

The first thing we need to do is set up the transaction portfolio api.

In [10]:
txn_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

We have also created a mapping file to create formatted headings for the data that we receive back from LUSID.

We apply this mapping file to a dataframe showing each instrument, the unit balance and the cost basis. 

In [11]:
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

LusidInstrumentId SubHoldingKeys      InstrumentName SourcePortfolioId  \
0     LUID_KR3A1NMI             {}               Aviva         EQUITY_UK   
1     LUID_WSHJKJ2Y             {}                 BHP         EQUITY_UK   
2     LUID_SIMWQCNR             {}            Barclays         EQUITY_UK   
3     LUID_80DILFAS             {}                  BP         EQUITY_UK   
4     LUID_S1MNV9OQ             {}                HSBC         EQUITY_UK   
5     LUID_49KIZM5K             {}           Morrisons         EQUITY_UK   
6     LUID_AU5UQIVK             {}               Tesco         EQUITY_UK   
7     LUID_00IPL9KJ             {}           Rightmove         EQUITY_UK   
8     LUID_4C90VUEA             {}            vodafone         EQUITY_UK   
9     LUID_Y92ZIAH5             {}  Anglo American plc         EQUITY_UK   

  SourcePortfolioScope(default-Properties) HoldingType     Units  \
0                        38a9-cf1b-bb5a-5e           P  132000.0   
1                        38a9-cf1b-bb5a-5e           P  120000.0   
2                        38a9-cf1b-bb5a-5e           P  300000.0   
3                        38a9-cf1b-bb5a-5e           P  200000.0   
4                        38a9-cf1b-bb5a-5e           P   40000.0   
5                        38a9-cf1b-bb5a-5e           P  360000.0   
6                        38a9-cf1b-bb5a-5e           P   12000.0   
7                        38a9-cf1b-bb5a-5e           P  160000.0   
8                        38a9-cf1b-bb5a-5e           P  900000.0   
9                        38a9-cf1b-bb5a-5e           P   70000.0   

   SettledUnits  Amount-Cost Currency-Cost  Amount-CostPortfolioCcy  \
0      132000.0     660000.0           GBP                      0.0   
1      120000.0    2160000.0           GBP                      0.0   
2      300000.0     600000.0           GBP                      0.0   
3      200000.0    1000000.0           GBP                      0.0   
4       40000.0     240000.0           GBP                      0.0   
5      360000.0     720000.0           GBP                      0.0   
6       12000.0     100000.0           GBP                      0.0   
7      160000.0     960000.0           GBP                      0.0   
8      900000.0     900000.0           GBP                      0.0   
9       70000.0    1400000.0           GBP                      0.0   

  Currenct-CostPortfolioCcy  
0                       GBP  
1                       GBP  
2                       GBP  
3                       GBP  
4                       GBP  
5                       GBP  
6                       GBP  
7                       GBP  
8                       GBP  
9                       GBP

## Part 2: Adding Corporate Actions

We now want to process a selection of corporate actions that relate to instruments that we hold in our portfolio. 

We can apply these changes by processing them as transactions directly to the portfolio but this is not a scalable approach if you have these instruments held in multiple portfolios. 

We want to record the corporate actions in one place and have the option to apply them to every portfolio that holds the respective instruments. 

### Create a corporate actions source

The corporate actions source is a container for holding corporate actions that we can use to apply to any portfolio. 

You may want to use a different corporate action source for each of your data vendors.

In [12]:
ca_source_code = "ca_vendor1"

try:
    source_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_source_code,
        display_name=ca_source_code,
        description="Corporate Actions source for Output Transactions notebook",
    )
    
    source_result = api_factory.build(
        lusid.api.CorporateActionSourcesApi
    ).create_corporate_action_source(create_corporate_action_source_request=source_request)
    
except:
    pass

To attach the corporate action source to the portfolio you can use the TransactionPortfolioApi to upsert the details.

In [13]:
# Assign corporate actions source to our portfolio

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:00+00:00",
    create_portfolio_details=lusid.models.CreatePortfolioDetails(
      corporate_action_source_id=lusid.ResourceId(
          scope=scope,
          code=ca_source_code,
      ),  
    ),
)

print(
    f"Corporate actions source of {ca_source_code} assigned to portfolio {portfolio_code}"
)

Corporate actions source of ca_vendor1 assigned to portfolio EQUITY_UK


### Load transitions into a dataframe from an external source

Transitions determine which instrument is taking part in a corporate action, and what the effect of the corporate action is on holdings in that instrument.

In this example we are going to add transitions from a csv file for the following corporate actions:

* A bonus issue of 13,200 units from Aviva (1 bonus share per 10 held)
* A cash dividend of £204,000 from BHP (£1.70 GBP per share held)
* A 1:2 stock split from Tesco (each 1 share held is split into 2 shares held)

These transitions are posted with a payment date of 31 March 2020.

In [14]:
corporate_action_df = pd.read_csv("data/corp-acts/corpact_transitions_split.csv")

# Format the datetimes into ISO strings
for col in ["announcement_date", "ex_date", "payment_date", "record_date"]:
    corporate_action_df[col] = corporate_action_df[col].apply(
        lambda x: datetime.strptime(x, "%d/%m/%Y").strftime(format="%Y-%m-%dT00:00:00Z")
    )

corporate_action_df

code action_description description     announcement_date  \
0  5943592342      dividend-cash         BHP  2020-03-31T00:00:00Z   
1  5943592343        bonus-issue       Aviva  2020-03-31T00:00:00Z   
2  5943592344        stock-split       Tesco  2020-03-31T00:00:00Z   

                ex_date           record_date          payment_date client_id  \
0  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1235   
1  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1234   
2  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1240   

  input_instrument_name  input_units_factor  input_cost_factor  \
0                   BHP                   1                  1   
1                 Aviva                   1                  1   
2                 Tesco                   1                  1   

  output_instrument_internal  output_units_factor  output_cost_factor  \
0                    CCY_GBP                  1.7                   1   
1                    EQ_1234                  1.1                   1   
2                    EQ_1240                  2.0                   1   

   dividend_yield  
0             1.7  
1             0.0  
2             0.0

We need to add LUSID identifiers for the instruments onto the transitions.

In [15]:
# Add environment specific output LUIDs to the dataset

out_instrument_luid = []

for index, item in corporate_action_df.iterrows():
    if item["output_instrument_internal"].startswith("CCY_"):\
        out_instrument_luid.append("nan")
    else:
        out_instrument_luid.append(
            api_factory.build(lusid.api.InstrumentsApi)
            .get_instrument(
                identifier_type="ClientInternal",
                identifier=item["output_instrument_internal"],
            )
            .lusid_instrument_id
        )
        
corporate_action_df["output_instrument_luid"] = out_instrument_luid

In [16]:
# Add environment specific input LUIDs to the dataset
# Change the identifier_type as required - this can be any unique identifier which LUSID can resolve to a LUID

corporate_action_df["input_instrument_luid"] = corporate_action_df["client_id"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi)
    .get_instrument(identifier_type="ClientInternal", identifier=x)
    .lusid_instrument_id
)

You can see that the dataframe containing the transitions now have LUIDs that will enable LUSID to match up the input and output instruments for each transition to the instruments that we uploaded earlier in the notebook.

In [17]:
corporate_action_df

code action_description description     announcement_date  \
0  5943592342      dividend-cash         BHP  2020-03-31T00:00:00Z   
1  5943592343        bonus-issue       Aviva  2020-03-31T00:00:00Z   
2  5943592344        stock-split       Tesco  2020-03-31T00:00:00Z   

                ex_date           record_date          payment_date client_id  \
0  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1235   
1  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1234   
2  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z  2020-03-31T00:00:00Z   EQ_1240   

  input_instrument_name  input_units_factor  input_cost_factor  \
0                   BHP                   1                  1   
1                 Aviva                   1                  1   
2                 Tesco                   1                  1   

  output_instrument_internal  output_units_factor  output_cost_factor  \
0                    CCY_GBP                  1.7                   1   
1                    EQ_1234                  1.1                   1   
2                    EQ_1240                  2.0                   1   

   dividend_yield output_instrument_luid input_instrument_luid  
0             1.7                    nan         LUID_WSHJKJ2Y  
1             0.0          LUID_KR3A1NMI         LUID_KR3A1NMI  
2             0.0          LUID_AU5UQIVK         LUID_AU5UQIVK

### Translate the dataframe into  LUSID Transition

We want to take the transitions from the dataframe and break each up into their component input and output objects in preparation for uploading into LUSID.

In [18]:
# This code produces a dictionary of transitions where:
# The dict key is the corporate action code and the dict value is a list
# of input and output CorporateActionTransitionComponentRequest objects

transitions = {}
LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId"

for index, ca in corporate_action_df.iterrows():
    
    # Create transition components
    cat_in = models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={
            LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
        },
        units_factor=ca["input_units_factor"],
        cost_factor=ca["input_cost_factor"],
    )
    
    # Determine if the output is Cash or Stock, and create the 
    # appropriate transition component
    if str(ca["output_instrument_luid"]) == "nan":
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Currency": ca["output_instrument_internal"][4:]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
    else:
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
        
    key = ca["code"]
    transitions.setdefault(key, [])
    transitions[key].append(ca["action_description"])
    transitions[key].append(cat_in)
    transitions[key].append(cat_out)

Using the broken-down transitions we want to create an [UpsertCorporateActionRequest](https://www.lusid.com/docs/api/#operation/BatchUpsertCorporateActions) for each corporate action.

In [19]:
# Iterate through the transitions, turining them into LUSID Corporate Action Requests

for key, values in transitions.items():
    transition_code = key
    transition_type = values[0]
    transition_in = values[1]
    transitions_out = values[2:]
    
    for x in transitions_out:
        if isinstance(x, str):
            transitions_out.remove(x)
            
    temp_transition = models.CorporateActionTransition(
        input_transition=transition_in, output_transitions=transitions_out
    )
    
    # Extract the data for the corporate action from the LUID corporate actions dataframe
    data = corporate_action_df[corporate_action_df["code"] == transition_code]
    
    # Iterate through each row of data and create the Corporate Action
    # Request for the appropriate type of action.
    for row, item in data.iterrows():
        if transition_type == "dividend-cash":
            div_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        if transition_type == "bonus-issue":
            bonus_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        if transition_type == "stock-split":
            split_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        
            

### Upsert the corporate actions into LUSID's movements engine

The prior step left us with 3 variables that package all the information that LUSID needs to process the transitions:
* `div_ca`
* `bonus_ca`
* `split_ca`

These are fed into the CorporateActionSourcesApi as a batch upsert of corporate actions.

In [20]:
result = api_factory.build(
    lusid.api.CorporateActionSourcesApi
).batch_upsert_corporate_actions(
    scope=scope, 
    code=ca_source_code, 
    upsert_corporate_action_request=[div_ca, bonus_ca, split_ca],
)

### Check holdings to see if the corporate actions have had an impact

The portfolio should now have:
* 145,200 units of Aviva (+ 13,200 units)
* £204,000 GBP cash (+ £204,000 cash)
* 24,000 units of Tesco (+12,000 units)

In [21]:
response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

LusidInstrumentId SubHoldingKeys      InstrumentName SourcePortfolioId  \
0      LUID_KR3A1NMI             {}               Aviva         EQUITY_UK   
1      LUID_WSHJKJ2Y             {}                 BHP         EQUITY_UK   
2      LUID_SIMWQCNR             {}            Barclays         EQUITY_UK   
3      LUID_80DILFAS             {}                  BP         EQUITY_UK   
4      LUID_S1MNV9OQ             {}                HSBC         EQUITY_UK   
5      LUID_49KIZM5K             {}           Morrisons         EQUITY_UK   
6      LUID_AU5UQIVK             {}               Tesco         EQUITY_UK   
7      LUID_00IPL9KJ             {}           Rightmove         EQUITY_UK   
8      LUID_4C90VUEA             {}            vodafone         EQUITY_UK   
9      LUID_Y92ZIAH5             {}  Anglo American plc         EQUITY_UK   
10           CCY_GBP             {}             CCY_GBP         EQUITY_UK   

   SourcePortfolioScope(default-Properties) HoldingType     Units  \
0                         38a9-cf1b-bb5a-5e           P  145200.0   
1                         38a9-cf1b-bb5a-5e           P  120000.0   
2                         38a9-cf1b-bb5a-5e           P  300000.0   
3                         38a9-cf1b-bb5a-5e           P  200000.0   
4                         38a9-cf1b-bb5a-5e           P   40000.0   
5                         38a9-cf1b-bb5a-5e           P  360000.0   
6                         38a9-cf1b-bb5a-5e           P   24000.0   
7                         38a9-cf1b-bb5a-5e           P  160000.0   
8                         38a9-cf1b-bb5a-5e           P  900000.0   
9                         38a9-cf1b-bb5a-5e           P   70000.0   
10                        38a9-cf1b-bb5a-5e           B  204000.0   

    SettledUnits  Amount-Cost Currency-Cost  Amount-CostPortfolioCcy  \
0       145200.0     660000.0           GBP                      0.0   
1       120000.0    2160000.0           GBP                      0.0   
2       300000.0     600000.0           GBP                      0.0   
3       200000.0    1000000.0           GBP                      0.0   
4        40000.0     240000.0           GBP                      0.0   
5       360000.0     720000.0           GBP                      0.0   
6        24000.0     100000.0           GBP                      0.0   
7       160000.0     960000.0           GBP                      0.0   
8       900000.0     900000.0           GBP                      0.0   
9        70000.0    1400000.0           GBP                      0.0   
10      204000.0     204000.0           GBP                      0.0   

   Currenct-CostPortfolioCcy  
0                        GBP  
1                        GBP  
2                        GBP  
3                        GBP  
4                        GBP  
5                        GBP  
6                        GBP  
7                        GBP  
8                        GBP  
9                        GBP  
10                       GBP

## Part 3: Manual holding adjustment

It is possible to manually adjust instrument holdings in LUSID. You may wish to do this if you want to make a quick fix for an incorrect position or if you know that there is a mismatch between some of your systems.

In time you would look to properly account for any mismatch.

### Adjust holdings to add an initial cash balance

Seeing the cash increase in the portfolio has highlighted the fact that there was no initial cash position on record. 

We can manually set the holdings of our portfolio to add in a starting cash balance of £500,000 GBP. We will make this effective from the same time the portfolio was created.

In [22]:
# Set the date from which the cash balance will apply to be the start of the portfolio
response = txn_portfolio_api.get_details(
    scope=scope,
    code=portfolio_code,
)

holdings_effective_date = response.version.effective_from

# Define the actual initial cash balance
initial_cash_balance = 500000

# Create a holding adjustment to set our initial cash balance
holding_adjustment = [
    models.AdjustHoldingRequest(
        instrument_identifiers={
            "Instrument/default/Currency": "GBP"},
        tax_lots=[
            models.TargetTaxLotRequest(
                units=initial_cash_balance,
            )
        ]
    )
]

Once we have the adjustment set up, we can use the TransactionPortfoliosApi to update the portfolio in LUSID.

In [23]:
set_holdings_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at=holdings_effective_date,
    adjust_holding_request=holding_adjustment,
)

### Check holdings post-adjustment

The portfolio should now have:
* £704,000 GBP cash (+ £500,000 cash)


In [24]:
response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

LusidInstrumentId SubHoldingKeys      InstrumentName SourcePortfolioId  \
0      LUID_KR3A1NMI             {}               Aviva         EQUITY_UK   
1      LUID_WSHJKJ2Y             {}                 BHP         EQUITY_UK   
2      LUID_SIMWQCNR             {}            Barclays         EQUITY_UK   
3      LUID_80DILFAS             {}                  BP         EQUITY_UK   
4      LUID_S1MNV9OQ             {}                HSBC         EQUITY_UK   
5      LUID_49KIZM5K             {}           Morrisons         EQUITY_UK   
6      LUID_AU5UQIVK             {}               Tesco         EQUITY_UK   
7      LUID_00IPL9KJ             {}           Rightmove         EQUITY_UK   
8      LUID_4C90VUEA             {}            vodafone         EQUITY_UK   
9      LUID_Y92ZIAH5             {}  Anglo American plc         EQUITY_UK   
10           CCY_GBP             {}             CCY_GBP         EQUITY_UK   

   SourcePortfolioScope(default-Properties) HoldingType     Units  \
0                         38a9-cf1b-bb5a-5e           P  145200.0   
1                         38a9-cf1b-bb5a-5e           P  120000.0   
2                         38a9-cf1b-bb5a-5e           P  300000.0   
3                         38a9-cf1b-bb5a-5e           P  200000.0   
4                         38a9-cf1b-bb5a-5e           P   40000.0   
5                         38a9-cf1b-bb5a-5e           P  360000.0   
6                         38a9-cf1b-bb5a-5e           P   24000.0   
7                         38a9-cf1b-bb5a-5e           P  160000.0   
8                         38a9-cf1b-bb5a-5e           P  900000.0   
9                         38a9-cf1b-bb5a-5e           P   70000.0   
10                        38a9-cf1b-bb5a-5e           B  704000.0   

    SettledUnits  Amount-Cost Currency-Cost  Amount-CostPortfolioCcy  \
0       145200.0     660000.0           GBP                      0.0   
1       120000.0    2160000.0           GBP                      0.0   
2       300000.0     600000.0           GBP                      0.0   
3       200000.0    1000000.0           GBP                      0.0   
4        40000.0     240000.0           GBP                      0.0   
5       360000.0     720000.0           GBP                      0.0   
6        24000.0     100000.0           GBP                      0.0   
7       160000.0     960000.0           GBP                      0.0   
8       900000.0     900000.0           GBP                      0.0   
9        70000.0    1400000.0           GBP                      0.0   
10      704000.0     204000.0           GBP                      0.0   

   Currenct-CostPortfolioCcy  
0                        GBP  
1                        GBP  
2                        GBP  
3                        GBP  
4                        GBP  
5                        GBP  
6                        GBP  
7                        GBP  
8                        GBP  
9                        GBP  
10                       GBP

## Part 4: What transactions make up our holdings?

Now we have the portfolio holdings looking correct thanks to a combination of initial transactions, a few corporate actions and a manual holding adjustment. 

LUSID calculates holdings and valuations based on a timeseries of transactions, so what has it done to include the corporate actions and the adjustment?

Lets extract the list of transactions from LUSID to examine them a little closer.


In [25]:
# Load a mapping file for DataFrame headers for the build transaction response
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

# Query LUSID for the transactions that form our portfolio
build_transactions_response = txn_portfolio_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date="2020-01-01", end_date="2020-12-31"
    ),
    property_keys=["Instrument/default/Name"],
)

build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)
build_transactions_df

TransactionId     TransactionType  \
0                            trd_0001             StockIn   
1                            trd_0002             StockIn   
2                            trd_0003             StockIn   
3                            trd_0004             StockIn   
4                            trd_0005             StockIn   
5                            trd_0006             StockIn   
6                            trd_0007             StockIn   
7                            trd_0008             StockIn   
8                            trd_0009             StockIn   
9                            trd_0010             StockIn   
10                           trd_0011             StockIn   
11                           trd_0012             StockIn   
12                           trd_0022             StockIn   
13                           trd_0015             StockIn   
14                           trd_0016             StockIn   
15                           trd_0017             StockIn   
16                           trd_0018             StockIn   
17                           trd_0019             StockIn   
18                           trd_0020             StockIn   
19                           trd_0014             StockIn   
20                         5943592344     CorpActIncrease   
21                         5943592343     CorpActIncrease   
22  2020-01-01T00:00:00.0000000+00:00  AdjustmentIncrease   
23                         5943592342            Dividend   

          TransactionTypeDesc ClientId LusidInstrumentId  \
0                 Transfer In  EQ_1234     LUID_KR3A1NMI   
1                 Transfer In  EQ_1234     LUID_KR3A1NMI   
2                 Transfer In  EQ_1235     LUID_WSHJKJ2Y   
3                 Transfer In  EQ_1235     LUID_WSHJKJ2Y   
4                 Transfer In  EQ_1236     LUID_SIMWQCNR   
5                 Transfer In  EQ_1236     LUID_SIMWQCNR   
6                 Transfer In  EQ_1237     LUID_80DILFAS   
7                 Transfer In  EQ_1237     LUID_80DILFAS   
8                 Transfer In  EQ_1238     LUID_S1MNV9OQ   
9                 Transfer In  EQ_1238     LUID_S1MNV9OQ   
10                Transfer In  EQ_1239     LUID_49KIZM5K   
11                Transfer In  EQ_1239     LUID_49KIZM5K   
12                Transfer In  EQ_1240     LUID_AU5UQIVK   
13                Transfer In  EQ_1241     LUID_00IPL9KJ   
14                Transfer In  EQ_1241     LUID_00IPL9KJ   
15                Transfer In  EQ_1242     LUID_4C90VUEA   
16                Transfer In  EQ_1242     LUID_4C90VUEA   
17                Transfer In  EQ_1243     LUID_Y92ZIAH5   
18                Transfer In  EQ_1243     LUID_Y92ZIAH5   
19                Transfer In  EQ_1240     LUID_AU5UQIVK   
20  Corporate Action Increase      NaN     LUID_AU5UQIVK   
21  Corporate Action Increase      NaN     LUID_KR3A1NMI   
22        Increase Adjustment      NaN           CCY_GBP   
23           Dividend Payment      NaN           CCY_GBP   

             TransactionDate            SettlementDate     Units  \
0  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00  120000.0   
1  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00   12000.0   
2  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00   60000.0   
3  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00   60000.0   
4  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00  150000.0   
5  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00  150000.0   
6  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00  100000.0   
7  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00  100000.0   
8  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00   20000.0   
9  2020-01-02 00:00:00+00:00 2020-01-04 00:00:00+00:00   20000.0   
10 2020-01-15 00:00:00+00:00 2020-01-17 00:00:00+00:00  180000.0   
11 2020-01-15 00:00:00+00:00 2020-01-17 00:00:00+00:00  180000.0   
12 2020-01-15 00:00:00+00:00 2020-01-17 00:00:00+00:00    8000.0   
13 2020-01-15 00:00:00+00:00 2020-01-17 00:00

We initialised the portfolio with 20 'StockIn' transactions which are visible as the first 20 items in the Output Transactions list. 

The remaining four transactions that we can see in the output are 'synthetic transactions' that LUSID creates in our portfolio as a result of the actions and adjustments that we have instructed. 

LUSID is built to ensure that anything that impacts a portfolio is represented, even if it has not been directly applied.

### Some things to note with transaction IDs

LUSID needs to create transaction IDs for synthetic transactions and rather than creating unique identifiers each time, it is designed to use IDs based on the source transaction, transition or action to retain the relationships and traceability. 

We can see this in our notebook by initially looking at the transaction IDs for the corporate actions in rows 20,21 and 22. The IDs here match the 'code' or ID of the transitions that they are based on. 

This is also the case for the ID of the holding adjustment, however, in this case the ID used is simply the date-time that the adjustment was made. 

The general pattern is that there is a single 'input transaction' that has a unique identifier and all 'built transactions' relating to that input have the same identifier so that they can be linked. 

This means that if we pro-rate a coupon payment across three s-h-k the three transactions will have the same id. Alternatively, if we split a transaction via strategy tags, each will have the same id.